In [1]:
# !pip install oura-ring

In [2]:
import pandas as pd
import datetime

# import matplotlib.pyplot as plt
# import seaborn as sns
# sns.set_style('whitegrid')

from oura_ring import OuraClient
import gspread

from config import config

In [3]:
def excel_date(date1):
    """
    converts datetime date to excel int date
    https://stackoverflow.com/a/9574948
    """
    temp = datetime.datetime(1899, 12, 30)    # Note, not 31st Dec but 30th!
    delta = date1 - temp
    return float(delta.days) + (float(delta.seconds) / 86400)

In [4]:
f = open(config.oura_key, 'r').read()
client = OuraClient(f)

In [5]:
# if first time, run one month
tdy = datetime.datetime.strftime(
    datetime.datetime.today() - datetime.timedelta(days=0), 
    '%Y-%m-%d'
)
ydy = datetime.datetime.strftime(
    datetime.datetime.today() - datetime.timedelta(days=config.days), 
    '%Y-%m-%d'
)
ydy, tdy

('2022-11-27', '2022-12-27')

In [6]:
# # set timezone to local
# hr = client.get_heart_rate(ydy, tdy)
# hr = pd.DataFrame(hr).set_index('timestamp').drop(columns=['source'])
# hr.index = [pd.to_datetime(_.replace('+00:00', '')) for _ in hr.index]
# hr.index = hr.index.tz_localize('UTC')
# hr.index = hr.index.tz_convert('Asia/Seoul')
# hr.plot(figsize=(13,5));

In [7]:
# get all sleep for this week
sleep = client.get_sleep_periods(ydy, tdy)

In [8]:
# filter sleep to what we need and engineer duration
s = [
    {
        x: _[x] for x in _ if x in [
            'day',
            'bedtime_end', 
            'bedtime_start',
            'total_sleep_duration',
        ]
    } for _ in sleep
]
for _ in s:
    _['dur'] = pd.to_datetime(_['bedtime_end']) - pd.to_datetime(_['bedtime_start'])

# convert seconds to hours
total_sleep = pd.DataFrame(s)[['day', 'total_sleep_duration']].groupby('day').sum() / 60 / 60
total_sleep

,total_sleep_duration
day,
2022-11-28,7.925000
2022-11-29,8.716667
2022-11-30,5.700000
2022-12-01,8.975000
2022-12-03,0.116667
2022-12-04,9.000000
2022-12-05,8.250000
2022-12-06,8.508333
2022-12-10,7.325000


In [9]:
# get HRV
results = []
for s in sleep:
    hrv = None
    if s['heart_rate']:
        hr = pd.DataFrame(s['heart_rate']['items'])
        # hrv = ((60000 / hr).diff() ** 2).mean() ** 0.5
        # hrv = hrv.values[0]
    results.append({
        'start': s['bedtime_start'],
        'avg_hrv': s['average_hrv'],
        # 'calc_hrv': hrv,
        'day': s['day']
    })

avg_hrv = pd.DataFrame(results)[['day', 'avg_hrv']].groupby('day').mean()
avg_hrv

,avg_hrv
day,
2022-11-28,35.000000
2022-11-29,31.000000
2022-11-30,21.000000
2022-12-01,44.000000
2022-12-03,23.500000
2022-12-04,41.000000
2022-12-05,32.000000
2022-12-06,31.000000
2022-12-10,27.666667


In [10]:
# pd.DataFrame(results).set_index('start').plot();

In [11]:
# join sleep duration to HRV
to_write = total_sleep.join(avg_hrv).reset_index()
to_write['day'] = to_write['day'].apply(
    lambda x: excel_date(datetime.datetime.strptime(x, '%Y-%m-%d'))
)

In [12]:
# check what's currently in the sheet
gc = gspread.service_account(config.svc_acct_json)
sh = gc.open_by_key(config.sheet_id)
wks = sh.worksheet(config.tab_name)
data = wks.get_all_values()

# here's what is in the sheet already
data

[['day', 'total_sleep_duration', 'avg_hrv'],
 ['44,898.00', '0.1166666667', '23.5'],
 ['44,899.00', '9', '41'],
 ['44,900.00', '8.25', '32'],
 ['44,890.00', '7.616666667', '33'],
 ['44,893.00', '7.925', '35'],
 ['44,894.00', '8.716666667', '31'],
 ['44,895.00', '5.7', '21'],
 ['44,896.00', '8.975', '44'],
 ['44,898.00', '0.1166666667', '23.5'],
 ['44,899.00', '9', '41'],
 ['44,900.00', '8.25', '32'],
 ['44,901.00', '8.508333333', '31'],
 ['44,905.00', '7.325', '27.66666667'],
 ['44,906.00', '11.98333333', '35'],
 ['44,907.00', '6.933333333', '34'],
 ['44,908.00', '9.841666667', '33'],
 ['44,910.00', '8.358333333', '29.8'],
 ['44,911.00', '7.616666667', '25.8'],
 ['44,912.00', '11.36666667', '29.25'],
 ['44,913.00', '0.1666666667', '22'],
 ['44,914.00', '11.04166667', '23.66666667'],
 ['44,915.00', '8.266666667', '33'],
 ['44,916.00', '8.733333333', '27'],
 ['44,918.00', '7.041666667', '21.66666667'],
 ['44,919.00', '0.6083333333', '45'],
 ['44,920.00', '12.86666667', '32'],
 ['44,921.0

In [14]:
# if no data, write everything
if not len(data):
    
    wks.update(
        [to_write.columns.to_list()] +  to_write.values.tolist()
    )
    
# if data, only append what's not already in data
elif len(data):

    wks.append_rows(
        [_ for _ in to_write.values.tolist() 
        if _[0] not in [float(_[0].replace(',', '')) for _ in data[1:]]] # filters data to push only what's not in the sheet
    )